## Uczenie Nienadzorowane - projekt
- Sebastian Borukało (259188)
- Jakub Bukała (259189)
- Jakub Smakowski (259209)

Celem projektu jest zbudowanie modelu uzupełniającego (inpainting) uszkodzenia dzieł sztuki pochodzącyzh z platformy WikiArt w oparciu o metody uczenia nienadzorowanego (unsupervised) i samonadzorowanego (self-supervised) należy zbudować system pozwalający na grupowanie obrazów na podstawie ich podobieństwa oraz uzupełnianie ich symulowanych uszkodzeń.

1. Budowę systemu należy rozpocząć od budowy generatora uszkodzeń obrazów (białych plam /masek). Podstawowa wersja uszkadzania obrazów polega na losowym wycinaniu / maskowaniu kwadratowych fragmentów obrazu, zajmujących nie więcej niż 1/16 obrazu. Rozszerzona wersja powinna pozwalać na generowanie uszkodzeń o nieregularnych kształtach.
2. Kolejnym krokiem powinno być zbudowanie zredukowanej reprezentacji obrazów ze zbioru treningowego a następnie jej klasteryzacji w celu znalezienia naturalnego grupowania styli. Grupowanie należy następnie porównać z dostępnymi w zbiorze etykietami stylu i autora.
3. Następnie, dla każdej z grup należy nauczyć osobny model uzupełniający uszkodzenia. Alternatywnie, możliwe jest zbudowanie jednego modelu który na wejściu przyjmowałby uszkodzony obraz oraz identyfikator grupy.
4. Końcowym etapem będzie zbudowanie modelu zwiększającego rozdzielczość obrazów

# ETAP 1
...................

# ETAP KOŃCOWY: Super-Resolution  
Celem tego kroku projektu jest stworzenie modelu Super-Resolution (SR), który potrafi zwiększyć rozdzielczość obrazów dzieł sztuki (zbiór WikiArt) z zachowaniem detali malarskich (pociągnięcia pędzla, faktura płótna).

**Wykorzystane technologie:**  
- Model: SRCNN (Super-Resolution Convolutional Neural Network).  
- Dane: WikiArt (Obrazy artystyczne).  
- Środowisko: Kaggle (GPU P100/T4).  
- Śledzenie eksperymentów: Comet ML.  

**Opis modelu:**
SRCNN realizuje proces w trzech krokach:
1. **Ekstrakcja:** Warstwa 1 wyodrębnia cechy z obrazu LR.
2. **Mapowanie:** Warstwa 2 mapuje cechy na przestrzeń wysokiej rozdzielczości.
3. **Rekonstrukcja:** Warstwa 3 składa obraz końcowy HR.

**Stosowane metryki:**
* **PSNR (dB):** Podstawowa miara jakości rekonstrukcji obrazu (wyrażana w dB). Im wyższa, tym lepiej.
* **L1 Loss:** Funkcja straty używana podczas treningu. W przeciwieństwie do MSE, L1 rzadziej powoduje nadmierne "rozmycie" obrazu, co jest kluczowe przy dziełach sztuki.


### 🚀 Krok 1: Pobranie Zbioru Danych (HR)  
Wykorzystujemy bibliotekę datasets z Hugging Face do pobrania zbioru HR (huggan/wikiart).

In [ ]:
from datasets import load_dataset
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import json
from tqdm import tqdm # Dodajemy tqdm do śledzenia postępu w pętli

DATASET_NAME = "huggan/wikiart"

# Używamy ścieżek bezwzględnych.
SAVE_DIR = r"C:\Users\admin\Documents\SIIUM2\UN\wikiart_hr_images"
SPLIT_DIR = r"C:\Users\admin\Documents\SIIUM2\UN\data_splits"

# Tworzymy katalogi, jeśli nie istnieją
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(SPLIT_DIR, exist_ok=True)

print(f"Ładowanie pełnego zbioru danych: {DATASET_NAME}...")

try:
    # Wczytujemy cały zbiór danych (split="train")
    dataset = load_dataset(DATASET_NAME, split="train", cache_dir="./hf_cache")
    print(f"Zbiór danych załadowany. Całkowita liczba obrazów: {len(dataset)}")
except Exception as e:
    print(f"Błąd ładowania zbioru danych: {e}")
    exit()

image_paths = []
total_images = len(dataset)

print(f"Rozpoczynanie zapisu {total_images} obrazów na dysk przy użyciu pętli for...")

# Zapis Obrazów na Dysk
for index, example in enumerate(tqdm(dataset, total=total_images, desc="Zapisywanie obrazów HR")):
    try:
        img = example.get('image')
        
        if img is None:
            continue
            
        filename = os.path.join(SAVE_DIR, f"{index:06d}.png")
        
        if img.mode != 'RGB':
            img = img.convert('RGB')
            
        img.save(filename)
        image_paths.append(filename) # Dodajemy ścieżkę do listy
        
    except Exception as e:
        print(f"\nOstrzeżenie: Błąd zapisu obrazu {index}: {e}. Plik pominięty.")
        continue

print(f"\nZapis zakończony. Liczba poprawnie zapisanych plików: {len(image_paths)}")

### 🚀 Krok 2: Przetworzenie oryginalnych obrazów o wysokiej rozdzielczości do rozmiaru 512x512 i podział na zbiory: treningowy, walidacyjny i testowy (pliki JSON)  
Zastosowaliśmy Center Crop, aby uniknąć zniekształceń proporcji przy skalowaniu do kwadratu 512x512

In [ ]:
import os
import json
import multiprocessing
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# --- KONFIGURACJA ---
SOURCE_DIR = r"C:\Users\admin\Documents\SIIUM2\4_UN\wikiart_hr_images" # Źródło (duże pliki)
TARGET_DIR = r"C:\Users\admin\Documents\SIIUM2\4_UN\wikiart_hr_512" # Cel (małe pliki 512x512)
SPLIT_DIR = r"C:\Users\admin\Documents\SIIUM2\4_UN\data_splits_512" # Folder na JSONy

TARGET_SIZE = 512
TEST_SIZE = 0.15
VALIDATION_SIZE = 0.15 
# --------------------

os.makedirs(TARGET_DIR, exist_ok=True)
os.makedirs(SPLIT_DIR, exist_ok=True)

def process_image(filename):
    """Skaluje i przycina obraz, zwraca nazwę pliku jeśli sukces, None jeśli błąd"""
    src_path = os.path.join(SOURCE_DIR, filename)
    dst_path = os.path.join(TARGET_DIR, filename)
    
    # Optymalizacja: jeśli plik już jest, nie robimy go drugi raz, ale zwracamy nazwę
    if os.path.exists(dst_path):
        return filename

    try:
        with Image.open(src_path) as img:
            img = img.convert('RGB')
            w, h = img.size
            
            # Odrzucamy zbyt małe obrazy
            if w < TARGET_SIZE or h < TARGET_SIZE:
                return None
            
            # 1. Resize (skalowanie krótszego boku do 512)
            scale = TARGET_SIZE / min(w, h)
            new_w = int(w * scale)
            new_h = int(h * scale)
            img = img.resize((new_w, new_h), resample=Image.Resampling.LANCZOS)
            
            # 2. Center Crop (wycięcie środka)
            left = (new_w - TARGET_SIZE) // 2
            top = (new_h - TARGET_SIZE) // 2
            right = left + TARGET_SIZE
            bottom = top + TARGET_SIZE
            img = img.crop((left, top, right, bottom))
            
            img.save(dst_path, format='PNG')
            return filename
            
    except Exception:
        return None

if __name__ == '__main__':
    # 1. Pobranie listy plików źródłowych
    print("Skanowanie katalogu źródłowego...")
    all_files = [f for f in os.listdir(SOURCE_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"Znaleziono {len(all_files)} plików. Start przetwarzania...")
    
    successful_files = []

    # 2. Przetwarzanie równoległe
    num_workers = max(1, multiprocessing.cpu_count() - 2)
    
    with multiprocessing.Pool(num_workers) as pool:
        # imap_unordered zwraca wyniki w miarę ich kończenia
        for result in tqdm(pool.imap_unordered(process_image, all_files), total=len(all_files)):
            if result is not None:
                successful_files.append(result)

    print(f"\nPrzetworzono poprawnie: {len(successful_files)} obrazów.")

    # 3. Tworzenie podziału na zbiory (Train/Val/Test)
    print("Tworzenie plików JSON z podziałem na zbiory...")
    
    # Pełne ścieżki do nowych plików
    full_paths = [os.path.join(TARGET_DIR, f) for f in successful_files]

    # Podział: Najpierw wydzielamy Test
    train_val_paths, test_paths = train_test_split(
        full_paths, test_size=TEST_SIZE, random_state=42
    )

    # Podział: Resztę dzielimy na Train i Validation
    # Przeliczamy proporcję, bo pula się zmniejszyła o test
    val_split_ratio = VALIDATION_SIZE / (1.0 - TEST_SIZE)
    train_paths, val_paths = train_test_split(
        train_val_paths, test_size=val_split_ratio, random_state=42
    )

    splits = {
        "train": train_paths,
        "validation": val_paths,
        "test": test_paths
    }

    for split_name, paths_list in splits.items():
        file_path = os.path.join(SPLIT_DIR, f"{split_name}_paths.json")
        with open(file_path, 'w') as f:
            json.dump(paths_list, f)
        print(f" -> {split_name}: {len(paths_list)} obrazów zapisano w {file_path}")

    print("\nGOTOWE!")

### 🚀 Krok 3: Główny Trening na Kaggle (krok3 - kaggle notebook)
Cel: Serce projektu. Tutaj definiujemy model SRCNN, ładujemy dane, trenujemy sieć i logujemy wyniki do Comet ML.


In [ ]:
# Konfiguracja COMET
!pip install comet_ml

import comet_ml
from kaggle_secrets import UserSecretsClient

# Pobranie klucza z sekretów Kaggle
user_secrets = UserSecretsClient()
COMET_API_KEY = user_secrets.get_secret("COMET_API_KEY")

import os
from datetime import datetime, timezone, timedelta
current_time = (datetime.now(timezone.utc) + timedelta(hours=1)).strftime("%Y_%m_%d_%H%M")
unique_name = f"SR_{current_time}"

# Inicjalizacja eksperymentu
experiment = comet_ml.Experiment(
    api_key=COMET_API_KEY,
    project_name="wikiart-super-resolution",
    workspace="Smaku",
    auto_metric_logging=True
)

experiment.set_name(unique_name)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image
import json
import numpy as np
from tqdm.notebook import tqdm # tqdm wersja dla notebooków
import matplotlib.pyplot as plt
from torch.cuda.amp import GradScaler, autocast 

#Ścieżka do zdjęć
IMG_DIR = "/kaggle/input/huggan-wikiart-512x512/wikiart_hr_512" 
# Ścieżki do JSONów
TRAIN_JSON = "/kaggle/input/huggan-wikiart-512x512/data_splits_512/train_paths.json"
VAL_JSON = "/kaggle/input/huggan-wikiart-512x512/data_splits_512/validation_paths.json"

# --- KLASA DATASET ---
# Obsługuje konwersję ścieżek Windowsowych do Kaggle'owych oraz degradację w locie (LR z HR) przy użyciu Resize - BICUBIC
class WikiArtKaggleDataset(Dataset):
    def __init__(self, json_path, img_dir):
        with open(json_path, 'r') as f:
            self.raw_paths = json.load(f)
        
        self.img_dir = img_dir
        
        self.hr_transform = T.Compose([T.ToTensor()])
        # Degradacja w locie
        self.lr_transform = T.Compose([
            T.Resize((256, 256), interpolation=T.InterpolationMode.BICUBIC),
            T.ToTensor()
        ])

    def __len__(self):
        return len(self.raw_paths)

    def __getitem__(self, idx):
        # 1. Bierzemy starą ścieżkę z Windowsa
        windows_path = self.raw_paths[idx]
        
        # 2. Wyciągamy samą nazwę pliku (np. "00123.png")
        # Używamy ntpath, żeby obsłużyć slashe z Windowsa na Linuxie
        import ntpath
        filename = ntpath.basename(windows_path)
        
        # 3. Budujemy nową, poprawną ścieżkę na Kaggle
        full_path = os.path.join(self.img_dir, filename)
        
        try:
            img = Image.open(full_path).convert('RGB')
            return self.lr_transform(img), self.hr_transform(img)
        except Exception as e:
            # Fallback w razie błędu
            return self.__getitem__((idx + 1) % len(self))

# --- MODEL SRCNN ---        
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        # Etap 0: Powiększenie wstępne (Bicubic) do rozmiaru docelowego
        self.upsample = nn.Upsample(scale_factor=2, mode='bicubic', align_corners=False)
        # Etap 1: Ekstrakcja cech (Patch extraction and representation)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.relu1 = nn.ReLU(inplace=True)
        # Etap 2: Mapowanie nieliniowe (Non-linear mapping)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.relu2 = nn.ReLU(inplace=True)
        # Etap 3: Rekonstrukcja (Reconstruction)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)
        
    def forward(self, x):
        x = self.upsample(x) 
        out = self.relu1(self.conv1(x))
        out = self.relu2(self.conv2(out))
        out = self.conv3(out)
        return out
    

# Ustawienia
BATCH_SIZE = 64 
LEARNING_RATE = 1e-4
NUM_EPOCHS = 40
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Katalog wyjściowy (na Kaggle zapisujemy tylko do /kaggle/working/)
SAVE_DIR = "/kaggle/working/results"
os.makedirs(SAVE_DIR, exist_ok=True)

# Dataset i DataLoader
train_dataset = WikiArtKaggleDataset(TRAIN_JSON, IMG_DIR)
val_dataset = WikiArtKaggleDataset(VAL_JSON, IMG_DIR)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4,pin_memory=True)

# ... (inicjalizacja modelu) ...
model = SRCNN().to(DEVICE)
# criterion = nn.MSELoss() # Mean Squared Error
criterion = nn.L1Loss() # Mean Absolute Error
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scaler = torch.amp.GradScaler('cuda') # <--- (float32 -> float16)

# Scheduler: Zmniejszy LR o połowę (factor=0.5), jeśli przez 3 epoki (patience=3) wartość val_loss nie spadnie.
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',       # Reagujemy, gdy val_loss przestaje SPADAĆ
    factor=0.5,       # Mnożymy LR * 0.5 (zmniejszamy o połowę)
    patience=3,       # Czekamy 3 epoki zanim zareagujemy
    min_lr=1e-6       # Nie schodzimy poniżej tej wartości
)

def calculate_psnr(img1, img2, border=0):
    if border > 0:
        img1 = img1[..., border:-border, border:-border]
        img2 = img2[..., border:-border, border:-border]
        
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0: return 100
    return 10 * torch.log10(1. / mse)

# Logowanie parametrów do Cometa
experiment.log_parameters({
    "batch_size": BATCH_SIZE,
    "lr": LEARNING_RATE,
    "epochs": NUM_EPOCHS,
    "model": "SRCNN"
})

# --- WZNAWIANIE TRENINGU ---
START_EPOCH = 0
RESUME_PATH = "/kaggle/working/results/srcnn_epoch_10.pth" # Ścieżka do ostatniego zapisu

if os.path.exists(RESUME_PATH):
    print(f"Wczytuję wagi z: {RESUME_PATH}")
    model.load_state_dict(torch.load(RESUME_PATH))
    START_EPOCH = 10 # Ustawiamy, że zaczynamy od kolejnej epoki
else:
    print("Brak zapisu, zaczynam trening od zera.")

global_step = 0

for epoch in range(START_EPOCH,NUM_EPOCHS):
    model.train()

    # 1. Zmienna na sumowanie straty w tej epoce
    running_train_loss = 0.0
    
    loop = tqdm(train_loader, desc=f"Epoka {epoch+1}")
    
    for lr_imgs, hr_imgs in loop:
        lr_imgs, hr_imgs = lr_imgs.to(DEVICE), hr_imgs.to(DEVICE)
        
        optimizer.zero_grad()
        
        # --- Obliczenia w float16 ---
        with torch.amp.autocast('cuda'):
            outputs = model(lr_imgs)
            loss = criterion(outputs, hr_imgs)
        
        # 2. Dodajemy stratę z tego batcha do sumy
        running_train_loss += loss.item()
        
        # --- Skalowanie gradientów ---
        # Zamiast loss.backward() i optimizer.step() używamy scalera
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        loop.set_postfix(loss=loss.item())
        experiment.log_metric("train_loss_step", loss.item(), step=global_step, epoch=epoch+1)
        global_step += 1

    # 3. Obliczamy średnią
    avg_train_loss = running_train_loss / len(train_loader)
    
    # Walidacja
    model.eval()
    val_psnr = 0
    val_loss = 0  
    
    with torch.no_grad():
        for lr_imgs, hr_imgs in val_loader:
            lr_imgs, hr_imgs = lr_imgs.to(DEVICE), hr_imgs.to(DEVICE)
            
            # 1. Generujemy obraz
            outputs = model(lr_imgs)
            # 2. Liczymy stratę (Loss) dla walidacji
            loss = criterion(outputs, hr_imgs)
            val_loss += loss.item() # <--- Sumujemy stratę
            # 3. Liczymy PSNR
            val_psnr += calculate_psnr(outputs, hr_imgs, border=4).item()
    
    # Obliczamy średnie
    avg_val_loss = val_loss / len(val_loader) 
    avg_val_psnr = val_psnr / len(val_loader)
    
    print(f"Epoka {epoch+1} zakończona.")
    print(f" -> Train Loss (avg): {avg_train_loss:.6f}")
    print(f" -> Val Loss: {avg_val_loss:.6f}")
    print(f" -> Val PSNR: {avg_val_psnr:.2f} dB")
    
    # Logowanie metryki epoki do Cometa
    experiment.log_metric("train_loss_epoch", avg_train_loss, step=global_step, epoch=epoch+1)
    experiment.log_metric("val_psnr_epoch", avg_val_psnr, step=global_step, epoch=epoch+1)
    experiment.log_metric("val_loss_epoch", avg_val_loss, step=global_step, epoch=epoch+1)

    # Scheduler:
    # 1. Karmimy scheduler wynikiem walidacji (musi wiedzieć, czy jest postęp)
    scheduler.step(avg_val_loss)
    
    # 2. Pobieramy aktualny Learning Rate, żeby wysłać go do Cometa
    current_lr = optimizer.param_groups[0]['lr']
    experiment.log_metric("learning_rate", current_lr, step=global_step, epoch=epoch+1)
        
    # Zapis na dysku Kaggle
    model_path = f"{SAVE_DIR}/srcnn_epoch_{epoch+1}.pth"
    torch.save(model.state_dict(), model_path)
    
    # WYSYŁKA DO CHMURY COMET (Bezpieczna kopia)
    experiment.log_model(f"SRCNN_Epoch_{epoch+1}", model_path)
    
    # Wysyłanie przykładowego obrazu do Cometa
    # Wizualizacja porównawcza
    with torch.no_grad():
        # Bierzemy pierwszy obraz z batcha walidacyjnego (ostatni batch z pętli walidacji)
        
        # 1. Przygotowanie SR (Super Resolution - wynik modelu)
        sr_img = outputs[0].cpu().permute(1, 2, 0).numpy() # Z (C,H,W) na (H,W,C)
        sr_img = np.clip(sr_img, 0, 1) # Zabezpieczenie zakresu

        # 2. Przygotowanie HR (High Resolution - cel)
        hr_img = hr_imgs[0].cpu().permute(1, 2, 0).numpy()
        
        # 3. Przygotowanie LR (Low Resolution - wejście)
        lr_img_tensor = lr_imgs[0].cpu()
        # Resize, aby pasowało wymiarem do paska (LR jest mniejsze, trzeba je rozciągnąć do podglądu)
        lr_img_resized = T.Resize((hr_img.shape[0], hr_img.shape[1]), interpolation=T.InterpolationMode.BICUBIC)(lr_img_tensor)
        
        lr_img = lr_img_resized.permute(1, 2, 0).numpy()
        
        # 4. Sklejenie obrazów horyzontalnie: [Wejście] | [Model] | [Oryginał]
        combined_img = np.hstack((lr_img, sr_img, hr_img))
        
        # 5. Logowanie do Cometa
        experiment.log_image(combined_img, name=f"Porownanie_Epoka_{epoch+1}", step=global_step)

experiment.end()

### 📄 KROK 4: Wykorzystanie wytrenowanego modelu do zwiększenia rozdzielczości obrazu.

In [ ]:
import torch
import torch.nn as nn
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt

# --- KLASA SRCNN ---
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        # Etap 0: Powiększenie wstępne (Bicubic) do rozmiaru docelowego
        self.upsample = nn.Upsample(scale_factor=2, mode='bicubic', align_corners=False)

        # Etap 1: Ekstrakcja cech (Patch extraction and representation)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.relu1 = nn.ReLU(inplace=True)

        # Etap 2: Mapowanie nieliniowe (Non-linear mapping)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.relu2 = nn.ReLU(inplace=True)

        # Etap 3: Rekonstrukcja (Reconstruction)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)
        
    def forward(self, x):
        x = self.upsample(x) 
        out = self.relu1(self.conv1(x))
        out = self.relu2(self.conv2(out))
        out = self.conv3(out)
        return out

# --- FUNKCJA UPSCALINGU x4 W DWÓCH KROKACH ---
def upscale_x4_progressive(image_path, model_path, output_name="result_1024x1024.png"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 1. Ładowanie modelu
    print(f"Ładowanie modelu z: {model_path}")
    model = SRCNN().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # 2. Wczytanie obrazu (256x256)
    img = Image.open(image_path).convert('RGB')
    transform = T.ToTensor()
    # input_tensor: [1, 3, 256, 256]
    input_tensor = transform(img).unsqueeze(0).to(device)
    
    print(f"Start: {img.size}")

    with torch.no_grad():
        # --- KROK 1: 256 -> 512 ---
        print("Krok 1: Upscaling x2 (do 512x512)...")
        stage1_output = model(input_tensor)
        
        # Ważne: przycinamy wartości do 0-1 przed drugim krokiem, 
        # żeby model w drugim przebiegu dostał poprawne dane "obrazowe"
        stage1_output = stage1_output.clamp(0, 1)

        # --- KROK 2: 512 -> 1024 ---
        # Wynik kroku 1 wchodzi jako wejście do kroku 2!
        print("Krok 2: Upscaling x2 (do 1024x1024)...")
        final_output = model(stage1_output)
        
        final_output = final_output.clamp(0, 1)

    # 3. Zapis i wizualizacja
    result_img = T.ToPILImage()(final_output.squeeze(0).cpu())
    result_img.save(output_name)
    print(f"Gotowe! Wynikowy rozmiar: {result_img.size}")
    print(f"Zapisano jako: {output_name}")

    # Porównanie
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f"Input ({img.size})")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(result_img)
    plt.title(f"SRCNN x4 Output ({result_img.size})")
    plt.axis('off')
    plt.show()

# --- UŻYCIE ---
upscale_x4_progressive("042222-256x256.png", "srcnn_epoch_31.pth", output_name="result_1024x1024.png")

Podsumowanie: Model SRCNN wykazuje znaczną poprawę ostrości w porównaniu do standardowej interpolacji bicubic.